In [6]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader, load_index_from_storage, StorageContext, Settings, VectorStoreIndex

import faiss
from llama_index.vector_stores.faiss import FaissVectorStore

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

import torch
from llama_index.llms.huggingface import HuggingFaceLLM

from transformers import AutoTokenizer, AutoModelForCausalLM
from pathlib import Path

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("tinyllama-tokenizer")
model = AutoModelForCausalLM.from_pretrained("tinyllama-model")

# Function to get student outcomes and essays
def get_inputs():
    student_outcomes = input("Enter student outcomes (comma-separated): ").split(",")
    essays = []
    print("\nEnter student essays (press Enter on an empty line to submit):\n")
    while True:
        essay = input()
        if not essay:
            break
        essays.append(essay)
    return student_outcomes, essays

# Get student outcomes and essays
student_outcomes, essays = get_inputs()

# Initialize SimpleDirectoryReader with the path to your documents
reader = SimpleDirectoryReader(input_files=["C:/Users/User/Downloads/IOT.pdf"])
documents = reader.load_data()

# Initialize the parser
parser = SentenceSplitter.from_defaults(chunk_size=512, chunk_overlap=20)

# Parse documents into nodes
nodes = parser.get_nodes_from_documents(documents)

# Initialize Faiss index
faiss_index = faiss.IndexFlatL2(768)

# Load the embedding model
Settings.embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="stsb-distilbert-base"))

# Create a vector storage and its context
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Add the embeddings to the index
index = VectorStoreIndex(nodes, storage_context=storage_context)
index.storage_context.persist()

# Load index from storage
stored_index = load_index_from_storage(storage_context)

# Configure LLM
Settings.llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=512,
    generate_kwargs={"temperature": 0.1, "do_sample": False},
    tokenizer_name="tinyllama-tokenizer",
    model_name="tinyllama-model",
    tokenizer_kwargs={"max_length": 2048},
    model_kwargs={"torch_dtype": torch.float16}
)

# Function to calculate relevance score
def calculate_relevance(nodes, outcome):
    # Implement your relevance scoring logic here
    # For example, you can calculate relevance based on the number of relevant nodes
    relevant_nodes = [node for node in nodes if outcome in node.node.get_text()]
    relevance_score = len(relevant_nodes) / len(nodes) if len(nodes) > 0 else 0
    return relevance_score


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Enter student outcomes (comma-separated):  Focus on Assigned Topic: 4 pts Exemplary Entire essay is related to the assigned topic and allows the reader to understand much more about the topic. 3 pts Proficient Most of the essay is related to the assigned topic. The essay wanders off at one point, but the reader can still learn something about the topic. 2 pts Acceptable Some of the essay is related to the assigned topic, but a reader does not learn much about the topic. 1 pts Beginner No attempt has been made to relate the essay to the assigned topic., Reflection of Personal Learning: 4 pts Exemplary Shows great depth of knowledge and learning, reveals feelings and thoughts, abstract ideas reflected through use of specific details. 3 pts Proficient Relates learning with research and project, personal and general reflections included, uses concrete language. 2 pts Acceptable Does not go deeply into the reflection of learning, generalizations and limited insight, uses some detail. 1 pts 


Enter student essays (press Enter on an empty line to submit):



 Define the Internet of Things and describe its key components. Internet of Things is made out of hardware and software systems. Its key components are Sensors, which collects data and sends to the device.  Connectivity, it is the medium between devices to devices can be wifi or  Bluetooth. Next is cloud computing it provides a wide and cost efficient way  to handle large amounts of data that are being generated by IoT devices.
 What are the primary benefits of implementing IoT in business and industry? One example of the primary benefits of implementing Iot in businesses is that  it has enhanced customer experience, since IOT is very versatile it can provide  the customers their preferred preferences when it comes to the services that  they are applying to.
 Explain the concept of Edge Computing in IoT and discuss its significance. The importance of edge computing in the Internet of Things resides in its  capacity to get beyond cloud computing's drawbacks, especially in situations  wh

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Query the model with the essays
grades = {}
for outcome in student_outcomes:
    for essay in essays:
        query_engine = stored_index.as_query_engine()
        response = query_engine.query(essay)

        # Get the retrieved nodes
        nodes = response.source_nodes

        # Your grading code goes here, integrating with the retrieved nodes
        relevance_score = calculate_relevance(nodes, outcome)
        # Assign grades based on the relevance score
        if relevance_score >= 0.8:
            grades[(outcome, essay)] = "High"
        elif relevance_score >= 0.5:
            grades[(outcome, essay)] = "Medium"
        else:
            grades[(outcome, essay)] = "Low"

# Provide feedback based on the grades
for (outcome, essay), grade in grades.items():
    feedback = f"Outcome: {outcome}\nEssay: {essay}\nGrade: {grade}\nFeedback: "
    if grade == "High":
        feedback += "Well done! Your essay demonstrates excellent skills in this area."
    elif grade == "Medium":
        feedback += "Good effort! Your essay shows some skills in this area, but there is room for improvement."
    else:
        feedback += "There is room for improvement. Your essay lacks sufficient evidence of skills in this area."
    print(feedback)
